<a href="https://colab.research.google.com/github/mpasha7/PySpark_ProductsDB/blob/main/PySpark_ProductsDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Products').getOrCreate()
spark

In [3]:
products = spark.read.csv('Products.csv', header=True, inferSchema=True, sep=';', encoding='windows-1251')
products.show()

+---+------------+-----+-----+
| Id|        Name|Price|Count|
+---+------------+-----+-----+
|  1|      Огурцы|   50|   55|
|  2|      Томаты|   80|   67|
|  3|     Капуста|   30|   32|
|  4|      Яблоки|   50|   45|
|  5|   Апельсины|   70|   68|
|  6|    Виноград|   90|   53|
|  7|       Чипсы|  100|   12|
|  8|     Лимонад|   80|   24|
|  9|     Сендвич|  100|    5|
| 10|Салат Цезарь|  120|    7|
+---+------------+-----+-----+



In [4]:
products.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Count: integer (nullable = true)



In [5]:
categories = spark.read.csv('Categories.csv', header=True, inferSchema=True, sep=';', encoding='windows-1251')
categories.show()

+---+-----------------+
| Id|            Title|
+---+-----------------+
|  1|            Овощи|
|  2|           Фрукты|
|  3|             Мясо|
|  4|              Сыр|
|  5|             Хлеб|
|  6|   Весовые товары|
|  7| Акционные товары|
|  8| Импортные товары|
|  9|Готовая продукция|
+---+-----------------+



In [6]:
categories.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Title: string (nullable = true)



In [7]:
relations = spark.read.csv('Relations.csv', header=True, inferSchema=True, sep=';')
relations.show()

+---------+----------+
|ProductId|CategoryId|
+---------+----------+
|        1|         1|
|        1|         6|
|        2|         1|
|        2|         6|
|        3|         1|
|        3|         6|
|        4|         2|
|        4|         6|
|        4|         7|
|        5|         2|
|        5|         6|
|        6|         2|
|        6|         6|
|        9|         9|
|       10|         9|
|       10|         7|
+---------+----------+



In [8]:
relations.printSchema()

root
 |-- ProductId: integer (nullable = true)
 |-- CategoryId: integer (nullable = true)



In [9]:
def left_left_join(left_table,
                 many_to_many_table,
                 right_table,
                 on_cols: list,
                 output_cols: list):
    '''
    Связывает заданные датафреймы (left_table и right_table)
    сявзью "многие ко многим" через промежуточный датафрейм (many_to_many_table)
    по принципу LEFT JOIN + LEFT JOIN с заданными ключами (on_cols).
    Выбирает необходимые столбцы (output_cols) из полученной таблицы.
    '''
    return left_table.join(many_to_many_table, left_table[on_cols[0]] == many_to_many_table[on_cols[1]], how='left')\
                     .join(right_table, many_to_many_table[on_cols[2]] == right_table[on_cols[3]], how='left')\
                     .select(output_cols)

In [10]:
### Полученный датафрейм (result) содержит все пары "Имя продукта - Имя категории"
### и имена продуктов без категорий (Чипсы, Лимонад).
### Категории, не принадлежащие ни к одному продукту (Сыр, Хлеб, Импортные товары) в выборку не попали.
result = left_left_join(products, relations, categories,
                        ['Id', 'ProductId', 'CategoryId', 'Id'],
                        [products.Name, categories.Title])
result.show()

+------------+-----------------+
|        Name|            Title|
+------------+-----------------+
|      Огурцы|   Весовые товары|
|      Огурцы|            Овощи|
|      Томаты|   Весовые товары|
|      Томаты|            Овощи|
|     Капуста|   Весовые товары|
|     Капуста|            Овощи|
|      Яблоки| Акционные товары|
|      Яблоки|   Весовые товары|
|      Яблоки|           Фрукты|
|   Апельсины|   Весовые товары|
|   Апельсины|           Фрукты|
|    Виноград|   Весовые товары|
|    Виноград|           Фрукты|
|       Чипсы|             NULL|
|     Лимонад|             NULL|
|     Сендвич|Готовая продукция|
|Салат Цезарь| Акционные товары|
|Салат Цезарь|Готовая продукция|
+------------+-----------------+

